In [72]:
import torch
import time
import non_local_boxes

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Basic Gradient Descent

## 1.1. Maximization under constraints

As we maximize under constraints, we need the orthogonal projection onto the unit hypercube $[0,1]^{32}$:

In [46]:
def projected_wiring_basic(W):  # W is a 32xn tensor
    W = torch.maximum(W, torch.zeros_like(W))  # it outputs the element-wise maximum
    W = torch.minimum(W, torch.ones_like(W))   # similarly for minimum
    return W

In [ ]:
def projected_wiring(W):  # W is a 32xn tensor
    W = torch.maximum(W, torch.zeros_like(W))  # it outputs the element-wise maximum
    W = torch.minimum(W, torch.ones_like(W))   # similarly for minimum

    # Then we verify the 4 equalities of (5)
    for alpha in range(non_local_boxes.evaluate.nb_columns):
        #1
        if abs(W[0, alpha] - W[1, alpha]) <= abs(W[8, alpha] - W[9, alpha]):
            W[0, alpha] = (W[0, alpha]+W[1, alpha])/2
            W[1, alpha] = W[0, alpha]
        else:
            W[8, alpha] = (W[8, alpha]+W[9, alpha])/2
            W[9, alpha] = W[8, alpha]
            
        #2
        if abs(W[2, alpha] - W[3, alpha]) <= abs(W[10, alpha] - W[11, alpha]):
            W[2, alpha] = (W[2, alpha]+W[3, alpha])/2
            W[3, alpha] = W[2, alpha]
        else:
            W[10, alpha] = (W[10, alpha]+W[11, alpha])/2
            W[11, alpha] = W[10, alpha]
            
        #3
        if abs(W[4, alpha] - W[5, alpha]) <= abs(W[12, alpha] - W[13, alpha]):
            W[4, alpha] = (W[4, alpha]+W[5, alpha])/2
            W[5, alpha] = W[4, alpha]
        else:
            W[12, alpha] = (W[12, alpha]+W[13, alpha])/2
            W[13, alpha] = W[12, alpha]
                
        #4
        if abs(W[6, alpha] - W[7, alpha]) <= abs(W[14, alpha] - W[15, alpha]):
            W[6, alpha] = (W[6, alpha]+W[7, alpha])/2
            W[7, alpha] = W[6, alpha]
        else:
            W[14, alpha] = (W[14, alpha]+W[15, alpha])/2
            W[15, alpha] = W[14, alpha]

    return W

In [109]:
W = non_local_boxes.utils.random_wiring(3).detach()
torch.abs(W[0,:]-W[1,:]) <= torch.ones(3)/2

0.778342604637146

In [106]:
if False:
    print("Hello")

## 1.2. Gradient Descent

In [84]:
# Initialization
starting_W = non_local_boxes.utils.random_wiring(non_local_boxes.evaluate.nb_columns)
nb_steps = 20
learning_rate = 2

P = non_local_boxes.utils.SR
Q = non_local_boxes.utils.PR
external_grad = torch.ones(non_local_boxes.evaluate.nb_columns)


# Gradient Descent
print("Iterating...")
tic = time.time()
#W = torch.clone(starting_W)
W = starting_W
for i in range(nb_steps):
    non_local_boxes.evaluate.phi_flat(W, P, Q).backward(gradient=external_grad)
    W = projected_wiring(W + learning_rate*W.grad).detach()  # create a brand new tensor, forgeting the previous gradient
    W.requires_grad=True
toc = time.time()


# Result
list = non_local_boxes.evaluate.phi_flat(W, P, Q).detach().numpy()
print("")
print("Number of tested wirings: ", len(list))
print("--> Min: ", min(list))
print("--> Av.: ", sum(list)/len(list))
print("--> Max: ", max(list))
print("Duration: ", (toc-tic)*1e3, "ms")

Iterating...

Number of tested wirings:  1000
--> Min:  0.74883735
--> Av.:  0.7967481325864791
--> Max:  1.0
Duration:  201.96199417114258 ms
